### Neuron Network From Scratch 

In [1]:
import numpy as np

# Define the neural network components (as provided in the question)
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)


class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


class Optimizer_GD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def update_params(self, layer):
        layer.weights -= self.learning_rate * layer.dweights
        layer.biases -= self.learning_rate * layer.dbiases


    

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features (150 samples x 4 features)
y = iris.target.reshape(-1, 1)  # Labels (150 samples)

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)  # Use sparse_output instead of sparse
y = encoder.fit_transform(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (120, 4) (120, 3)
Testing data shape: (30, 4) (30, 3)


In [41]:

# Training loop
dense1 = Layer_Dense(4, 5)  # Input layer (4 features) -> Hidden layer (5 neurons)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 3)  # Hidden layer (5 neurons) -> Output layer (3 classes)
activation2 = Activation_Softmax()

loss_function = Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD(learning_rate=0.01)

for epoch in range(2001):  # Train for 1000 epochs
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    # Compute loss
    loss = np.mean(loss_function.forward(activation2.output, y_train))
    # Compute accuracy
    predictions = np.argmax(activation2.output, axis=1)
    true_labels = np.argmax(y_train, axis=1)
    accuracy = np.mean(predictions == true_labels)

    # Print statistics every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.3f}, Accuracy: {accuracy:.3f}')

    # Backward pass
    loss_function.backward(activation2.output, y_train)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)


Epoch 0, Loss: 1.106, Accuracy: 0.333
Epoch 100, Loss: 1.074, Accuracy: 0.900
Epoch 200, Loss: 0.958, Accuracy: 0.658
Epoch 300, Loss: 0.741, Accuracy: 0.658
Epoch 400, Loss: 0.574, Accuracy: 0.667
Epoch 500, Loss: 0.485, Accuracy: 0.817
Epoch 600, Loss: 0.428, Accuracy: 0.908
Epoch 700, Loss: 0.382, Accuracy: 0.958
Epoch 800, Loss: 0.341, Accuracy: 0.967
Epoch 900, Loss: 0.303, Accuracy: 0.975
Epoch 1000, Loss: 0.269, Accuracy: 0.983
Epoch 1100, Loss: 0.240, Accuracy: 0.983
Epoch 1200, Loss: 0.215, Accuracy: 0.983
Epoch 1300, Loss: 0.194, Accuracy: 0.983
Epoch 1400, Loss: 0.177, Accuracy: 0.983
Epoch 1500, Loss: 0.163, Accuracy: 0.983
Epoch 1600, Loss: 0.151, Accuracy: 0.983
Epoch 1700, Loss: 0.142, Accuracy: 0.983
Epoch 1800, Loss: 0.133, Accuracy: 0.983
Epoch 1900, Loss: 0.126, Accuracy: 0.983
Epoch 2000, Loss: 0.120, Accuracy: 0.983


In [42]:
# Forward pass on test data
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

# Compute test loss and accuracy
test_loss = np.mean(loss_function.forward(activation2.output, y_test))
test_predictions = np.argmax(activation2.output, axis=1)
test_true_labels = np.argmax(y_test, axis=1)
test_accuracy = np.mean(test_predictions == test_true_labels)

print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_accuracy:.3f}')

Test Loss: 0.147, Test Accuracy: 0.967
